In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Step 1: Load Data
telemetry_data =pd.read_csv(
    'telemetry_for_operations_training_2.csv',
    parse_dates=['create_dt'],
    date_parser=lambda x: pd.to_datetime(x, errors='coerce')  # Handles coercion of invalid formats
)
labels_data = pd.read_csv("operations_labels_training.csv")
telemetry_data['create_dt']


<ipython-input-14-a6f4c3779092>:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  telemetry_data =pd.read_csv(


,create_dt
0,2023-09-29 08:12:46
1,2023-09-29 08:12:47
2,2023-09-29 08:12:49
3,2023-09-29 08:12:51
4,2023-09-29 08:12:53
...,...
516068,2023-10-26 08:16:26
516069,2023-10-26 08:16:31
516070,2023-10-26 08:16:36
516071,2023-10-26 08:16:41


In [16]:
import pandas as pd
import joblib

# Step 1: Load the trained model
rf_classifier = joblib.load('random_forest_model.pkl')

# Step 2: Load the Validation Data
validation_data = pd.read_csv(
    'telemetry_for_operations_validation.csv',
    parse_dates=['create_dt'],
    date_parser=lambda x: pd.to_datetime(x, errors='coerce')  # Handles coercion of invalid formats
)
# Step 3: Preprocess DateTime Columns




<ipython-input-16-230bd17d4208>:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  validation_data = pd.read_csv(


In [18]:
validation_data['create_dt']

,create_dt
0,2023-09-29 08:03:27
1,2023-09-29 08:03:28
2,2023-09-29 08:03:33
3,2023-09-29 08:03:34
4,2023-09-29 08:03:35
...,...
260106,2023-09-30 16:32:25
260107,2023-09-30 16:32:30
260108,2023-09-30 16:32:35
260109,2023-09-30 16:32:40


In [19]:

# Step 4: Check for any missing values after conversion
missing_dates = validation_data[validation_data['create_dt'].isna()]

# Print the rows with missing create_dt values
print("Rows with missing create_dt values:")
print(missing_dates)

# Step 5: Feature Engineering - Extract useful features from datetime columns
validation_data['hour'] = validation_data['create_dt'].dt.hour
validation_data['day_of_week'] = validation_data['create_dt'].dt.weekday
validation_data['month'] = validation_data['create_dt'].dt.month
validation_data['second'] = validation_data['create_dt'].dt.second  # Extract seconds for later use if needed

# Step 6: Define Features for Prediction
# Features: Choose telemetry metrics and the extracted datetime features
features_validation = validation_data[['speed_gps', 'accel_forward_nn', 'accel_braking_nn',
                                       'accel_angular_nn', 'accel_vertical_nn',
                                       'hour', 'day_of_week', 'month']]

# Step 7: Make Predictions
validation_data['predicted_operation_state'] = rf_classifier.predict(features_validation)

# Step 8: Map Predicted Values to Integer Codes as Specified
state_mapping = {'idle': 0, 'loading': 1, 'riding_loaded': 2, 'unloading_lift': 3, 'riding_empty': 5}
validation_data['predicted_operation_state'] = validation_data['predicted_operation_state'].map(state_mapping)

# Step 9: Retain the original create_dt with seconds for the output
validation_data['create_dt'] = validation_data['create_dt'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Step 10: Save Predictions to a New CSV File
# Include only the columns 'create_dt', 'mdm_object_name', and 'predicted_operation_state'
output_columns = ['create_dt', 'mdm_object_name', 'predicted_operation_state']
validation_data[output_columns].to_csv('predicted_validation_data.csv', index=False, lineterminator='\r\n')

print("Predictions saved to 'predicted_validation_data.csv'")

Rows with missing create_dt values:
       create_dt  mdm_object_name  mdm_model_id       lon       lat  alt  \
60902        NaT             1492            62  0.017198  0.006647   39   
102168       NaT             1425            62  0.014344  0.002874  -37   
184505       NaT             1035            62 -0.040465  0.011229  194   
201253       NaT              125            61  0.024091 -0.021479  123   

        speed_gps  direction  accel_forward_nn  accel_braking_nn  \
60902         0.0       42.0             0.000             0.000   
102168       19.9        0.0             0.686             0.000   
184505       23.2      170.0             0.000             0.294   
201253       17.1       69.0             0.980             0.000   

        accel_angular_nn  accel_vertical_nn  
60902                0.0                0.0  
102168               0.0                0.0  
184505               0.0                0.0  
201253               0.0                0.0  
Predictions 

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Step 1: Load the Datasets
telemetry_data = pd.read_csv(
    'telemetry_for_operations_training_2.csv',
    parse_dates=['create_dt'],
    date_parser=lambda x: pd.to_datetime(x, errors='coerce')
)

labels_data = pd.read_csv("operations_labels_training.csv")

# Step 2: Preprocess DateTime Columns
telemetry_data['create_dt'] = pd.to_datetime(telemetry_data['create_dt'], errors='coerce')
labels_data['start_time'] = pd.to_datetime(labels_data['start_time'], errors='coerce')
labels_data['end_time'] = pd.to_datetime(labels_data['end_time'], errors='coerce')

# Step 3: Create operation_state column based on time intervals
telemetry_data['operation_kind_id'] = None

for _, label_row in labels_data.iterrows():
    mask = (
        (telemetry_data['create_dt'] >= label_row['start_time']) &
        (telemetry_data['create_dt'] <= label_row['end_time']) &
        (telemetry_data['mdm_object_name'] == label_row['mdm_object_name'])
    )
    telemetry_data.loc[mask, 'operation_kind_id'] = label_row['operation_kind_id']

# Drop rows with missing operation states
telemetry_data = telemetry_data.dropna(subset=['operation_kind_id'])

# Step 4: Feature Engineering with .loc to avoid SettingWithCopyWarning
telemetry_data.loc[:, 'hour'] = telemetry_data['create_dt'].dt.hour
telemetry_data.loc[:, 'day_of_week'] = telemetry_data['create_dt'].dt.weekday
telemetry_data.loc[:, 'month'] = telemetry_data['create_dt'].dt.month

# Selected features for the LSTM model
features = telemetry_data[['speed_gps', 'accel_forward_nn', 'accel_braking_nn',
                           'accel_angular_nn', 'accel_vertical_nn', 'hour',
                           'day_of_week', 'month']]

# Step 5: Normalize Feature Values
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Step 6: Create Sequences of Data
sequence_length = 30  # Number of time steps to consider in each sequence

def create_sequences(features, labels, seq_length):
    X, y = [], []
    for i in range(len(features) - seq_length):
        X.append(features[i:i + seq_length])
        y.append(labels[i + seq_length])
    return np.array(X), np.array(y)

# Use operation_kind_id directly as labels
labels = telemetry_data['operation_kind_id'].values

X, y = create_sequences(scaled_features, labels, sequence_length)

# Step 7: Split Data into Training and Testing Sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Build the LSTM Model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))  # 6 because classes are 0, 1, 2, 3, 5

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 9: Train the Model
checkpoint = ModelCheckpoint('lstm_model.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3,
                    batch_size=128, callbacks=[early_stopping, checkpoint])

# Step 10: Save the Model
model.save('/content/drive/MyDrive/OES/lstm_model.keras')
print(f"Model Evaluation: {model.evaluate(X_test, y_test)}")


<ipython-input-11-d4758a061414>:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  telemetry_data = pd.read_csv(
<ipython-input-11-d4758a061414>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telemetry_data.loc[:, 'hour'] = telemetry_data['create_dt'].dt.hour
<ipython-input-11-d4758a061414>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telemetry_data

Epoch 1/3
3175/3175 ━━━━━━━━━━━━━━━━━━━━ 35s 10ms/step - accuracy: 0.6900 - loss: 0.7990 - val_accuracy: 0.7698 - val_loss: 0.5821
Epoch 2/3
3175/3175 ━━━━━━━━━━━━━━━━━━━━ 38s 9ms/step - accuracy: 0.7716 - loss: 0.5773 - val_accuracy: 0.7864 - val_loss: 0.5352
Epoch 3/3
3175/3175 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.7926 - loss: 0.5274 - val_accuracy: 0.8085 - val_loss: 0.4809
3175/3175 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8079 - loss: 0.4819
Model Evaluation: [0.4809390604496002, 0.8085324168205261]


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Step 1: Load the pre-trained model
model = load_model('lstm_model.keras')

# Step 2: Load Training Data and Fit Scaler
# Load your training data
training_data = pd.read_csv('operations_label_training.csv')

# Assuming your training features are the same as your validation features
# Adjust this part based on your actual training features
training_features = training_data[['speed_gps', 'accel_forward_nn', 'accel_braking_nn',
                                   'accel_angular_nn', 'accel_vertical_nn', 'hour',
                                   'day_of_week', 'month']]

# Fit the scaler on training features
scaler = MinMaxScaler()
scaler.fit(training_features)

# Step 3: Load and Preprocess Validation Data
validation_data = pd.read_csv(
    'telemetry_for_operations_validation.csv',
    parse_dates=['create_dt'],
    date_parser=lambda x: pd.to_datetime(x, errors='coerce')
)

# Preprocess validation data similar to training data
validation_data['create_dt'] = pd.to_datetime(validation_data['create_dt'], errors='coerce')
validation_data['hour'] = validation_data['create_dt'].dt.hour
validation_data['day_of_week'] = validation_data['create_dt'].dt.weekday
validation_data['month'] = validation_data['create_dt'].dt.month

# Step 4: Extract and Normalize Validation Features
validation_features = validation_data[['speed_gps', 'accel_forward_nn', 'accel_braking_nn',
                                       'accel_angular_nn', 'accel_vertical_nn', 'hour',
                                       'day_of_week', 'month']]

# Normalize validation features using the fitted scaler from training data
validation_features = scaler.transform(validation_features)

# Step 5: Create Sequences for Validation Data
sequence_length = 30  # This should match the sequence length used during training

def create_sequences(features, seq_length):
    X = []
    for i in range(len(features) - seq_length):
        X.append(features[i:i + seq_length])
    return np.array(X)

X_val = create_sequences(validation_features, sequence_length)

# Step 6: Make Predictions on Validation Data
validation_predictions = model.predict(X_val)

# Convert predictions to label indices using `np.argmax`
predicted_label_indices = np.argmax(validation_predictions, axis=1)

# Ensure that the predicted indices are integers
predicted_label_indices = predicted_label_indices.astype(int)

# Step 7: Insert the predicted label indices into the validation dataframe
validation_data['predicted_operation_state'] = np.nan

# Align the predicted labels with the original data considering sequence_length offset
predicted_indices = range(sequence_length, sequence_length + len(predicted_label_indices))
validation_data.loc[predicted_indices, 'predicted_operation_state'] = predicted_label_indices

# Step 8: Fill NaN values if required (optional)
validation_data['predicted_operation_state'].fillna(-1, inplace=True)  # Or any placeholder

# Step 9: Save Results to CSV
validation_data[['create_dt', 'mdm_object_name', 'predicted_operation_state']].to_csv(
    'predicted_validation_data_lstm_encoded.csv', index=False
)

# Optional: Display a sample of the mapped predictions to ensure alignment
print(validation_data[['create_dt', 'mdm_object_name', 'predicted_operation_state']].head(40))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/OES/operations_label_training.csv'